## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`



mlflow server \
  --backend-store-uri sqlite:///backend.db \
  --default-artifact-root ./artifacts_local
  --port 5000


In [4]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [5]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1759132600643, experiment_id='0', last_update_time=1759132600643, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(sk_model=lr,artifact_path="models",input_example=X[:1])
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/09/29 10:06:22 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/09/29 10:06:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


default artifacts URI: '/home/davidmouquet/Bureau/devops-mooc/02-experiment_tracking/running-mlflow-examples/artifacts_local/1/ebbba20be65d4307b729d7692d313773/artifacts'
🏃 View run skittish-loon-816 at: http://127.0.0.1:5000/#/experiments/1/runs/ebbba20be65d4307b729d7692d313773
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [ ]:
mlflow.search_experiments()

### Interacting with the model registry

In [8]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [9]:
client.search_registered_models()

[]

In [10]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/09/29 10:09:48 WARNING mlflow.tracking._model_registry.fluent: Run with id ebbba20be65d4307b729d7692d313773 has no artifacts at artifact path 'models', registering model based on models:/m-07851aa3a2114cc4a2e8216825090c85 instead
2025/09/29 10:09:48 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1759133388799, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1759133388799, metrics=None, model_id=None, name='iris-classifier', params=None, run_id='ebbba20be65d4307b729d7692d313773', run_link='', source='models:/m-07851aa3a2114cc4a2e8216825090c85', status='READY', status_message=None, tags={}, user_id='', version='1'>